In [18]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset
from torchvision.models import resnet18, ResNet18_Weights
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
import re
from tqdm import tqdm
from PIL import Image

df = pd.read_csv('kaggle_3m\data.csv')
missing = df[df['age_at_initial_pathologic'].isna()]['Patient']
missing

109    TCGA_HT_A61B
Name: Patient, dtype: object

In [23]:
def get_images(dir):
    images = []
    ids = []

    for subdir in os.listdir(dir):
        path = os.path.join(dir, subdir)
        pattern = r"TCGA_(CS|DU|FG|HT|EZ)_(\w{4})"
        match = re.search(pattern, path)

        if match:
            ids.append(match.group(2))

        if path.startswith("kaggle_3m\TCGA"):
            for image_name in os.listdir(path):
                
                if "mask" in image_name:
                    continue
                elif "TCGA_HT_A61B" in image_name:
                    continue
                else:
                    images.append(os.path.join(path, image_name))
    return images, ids

def get_labels(images, ids):
    ids = [id for image in images for id in ids if  id in image]
    df['num_id'] = df['Patient'].str.extract(r'([^_]+)$')
    labels = []
    for id in ids:
        age = df.loc[df['num_id'] == id, 'age_at_initial_pathologic'].values
        labels.append(age)
    return labels

class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
            
        if self.transform:
            image = self.transform(image)
        
        return image, label

def evaluate(model, loader, device):
    model.eval()
    total_error = 0
    count = 0
    with torch.no_grad():  
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) 
            total_error += torch.sum(torch.abs(outputs - labels))
            count += images.size(0)
    
    return total_error / count


In [21]:
images, ids = get_images('kaggle_3m')
labels = get_labels(images, ids)
for i in range(len(images)):
    images[i] = cv2.imread(images[i])
    images[i] = np.array(images[i])

transform = transforms.Compose([
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

dataset = CustomImageDataset(images, labels, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

size = len(loader.dataset)
train_size = int(0.7 * size)
val_size = int(0.15 * size)
test_size = size - train_size - val_size

train_set, val_set, test_set = random_split(loader.dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 1)

device = torch.device("cpu")
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 20


for epoch in range(epochs):
    model.train() 
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs.float(), labels.float())

        loss.backward()  
        optimizer.step()  

    MAE = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}, Mean Absolute Error: {MAE.item()}")

Epoch 1: 100%|██████████| 168/168 [07:03<00:00,  2.52s/it]


tensor([[30.],
        [34.],
        [37.],
        [48.],
        [38.],
        [54.],
        [30.],
        [31.],
        [28.],
        [40.],
        [57.],
        [61.],
        [59.],
        [31.],
        [33.],
        [38.]], dtype=torch.float64)
tensor([[ 7.4971],
        [ 7.4842],
        [11.2821],
        [15.2018],
        [10.3912],
        [17.8211],
        [15.5206],
        [11.0833],
        [13.8383],
        [12.1444],
        [17.8205],
        [17.0349],
        [14.8410],
        [ 8.5709],
        [14.3495],
        [ 9.6847]])
tensor([[54.],
        [57.],
        [59.],
        [53.],
        [27.],
        [43.],
        [41.],
        [29.],
        [22.],
        [58.],
        [30.],
        [53.],
        [70.],
        [61.],
        [67.],
        [48.]], dtype=torch.float64)
tensor([[18.2196],
        [17.5698],
        [18.4079],
        [14.1279],
        [-0.9401],
        [13.9089],
        [14.2185],
        [ 8.5673],
        [10.3382],


Epoch 2: 100%|██████████| 168/168 [05:53<00:00,  2.10s/it]


tensor([[30.],
        [34.],
        [37.],
        [48.],
        [38.],
        [54.],
        [30.],
        [31.],
        [28.],
        [40.],
        [57.],
        [61.],
        [59.],
        [31.],
        [33.],
        [38.]], dtype=torch.float64)
tensor([[16.0416],
        [21.5887],
        [29.2023],
        [23.3826],
        [27.5054],
        [27.6937],
        [15.3377],
        [14.2564],
        [17.3841],
        [16.9023],
        [30.5027],
        [31.3137],
        [27.8941],
        [19.6170],
        [25.0640],
        [17.7327]])
tensor([[54.],
        [57.],
        [59.],
        [53.],
        [27.],
        [43.],
        [41.],
        [29.],
        [22.],
        [58.],
        [30.],
        [53.],
        [70.],
        [61.],
        [67.],
        [48.]], dtype=torch.float64)
tensor([[23.4410],
        [28.8285],
        [34.6970],
        [31.0158],
        [ 2.8061],
        [24.2905],
        [28.0649],
        [12.6171],
        [13.1885],


Epoch 3: 100%|██████████| 168/168 [04:55<00:00,  1.76s/it]


...

tensor([[30.],
        [34.],
        [37.],
        [48.],
        [38.],
        [54.],
        [30.],
        [31.],
        [28.],
        [40.],
        [57.],
        [61.],
        [59.],
        [31.],
        [33.],
        [38.]], dtype=torch.float64)
tensor([[26.7859],
        [29.7497],
        [35.6113],
        [44.2712],
        [50.1125],
        [51.5239],
        [32.0412],
        [34.0784],
        [31.5843],
        [40.4283],
        [47.1458],
        [58.5990],
        [62.3280],
        [32.7333],
        [40.6776],
        [36.0447]])
tensor([[54.],
        [57.],
        [59.],
        [53.],
        [27.],
        [43.],
        [41.],
        [29.],
        [22.],
        [58.],
        [30.],
        [53.],
        [70.],
        [61.],
        [67.],
        [48.]], dtype=torch.float64)
tensor([[51.0544],
        [55.3356],
        [57.9156],
        [49.2628],
        [26.2052],
        [40.2323],
        [40.0193],
        [27.6176],
        [22.8105],


Epoch 19: 100%|██████████| 168/168 [05:03<00:00,  1.81s/it]


tensor([[30.],
        [34.],
        [37.],
        [48.],
        [38.],
        [54.],
        [30.],
        [31.],
        [28.],
        [40.],
        [57.],
        [61.],
        [59.],
        [31.],
        [33.],
        [38.]], dtype=torch.float64)
tensor([[29.6247],
        [34.7014],
        [36.1593],
        [43.9200],
        [51.5022],
        [52.7147],
        [32.9067],
        [31.7801],
        [31.9979],
        [38.7201],
        [50.4754],
        [59.1262],
        [63.8474],
        [30.8017],
        [43.3754],
        [35.3079]])
tensor([[54.],
        [57.],
        [59.],
        [53.],
        [27.],
        [43.],
        [41.],
        [29.],
        [22.],
        [58.],
        [30.],
        [53.],
        [70.],
        [61.],
        [67.],
        [48.]], dtype=torch.float64)
tensor([[52.2426],
        [55.2667],
        [62.1090],
        [47.2078],
        [25.8596],
        [38.5613],
        [40.4715],
        [27.1137],
        [23.1516],


Epoch 20: 100%|██████████| 168/168 [05:43<00:00,  2.05s/it]


tensor([[30.],
        [34.],
        [37.],
        [48.],
        [38.],
        [54.],
        [30.],
        [31.],
        [28.],
        [40.],
        [57.],
        [61.],
        [59.],
        [31.],
        [33.],
        [38.]], dtype=torch.float64)
tensor([[28.9128],
        [33.1246],
        [36.8260],
        [47.3222],
        [52.0226],
        [53.3593],
        [35.5737],
        [30.8013],
        [31.7417],
        [43.3802],
        [50.7536],
        [62.6994],
        [62.0712],
        [35.3292],
        [41.6393],
        [37.4848]])
tensor([[54.],
        [57.],
        [59.],
        [53.],
        [27.],
        [43.],
        [41.],
        [29.],
        [22.],
        [58.],
        [30.],
        [53.],
        [70.],
        [61.],
        [67.],
        [48.]], dtype=torch.float64)
tensor([[55.7828],
        [59.1282],
        [62.2863],
        [52.9000],
        [27.9047],
        [40.2417],
        [43.6211],
        [30.8499],
        [27.9161],


In [22]:
MAE = evaluate(model, test_loader, device)
print(f"Mean Absolute Error: {MAE.item()}")

tensor([[51.],
        [66.],
        [59.],
        [70.],
        [49.],
        [66.],
        [40.],
        [33.],
        [36.],
        [47.],
        [61.],
        [33.],
        [49.],
        [51.],
        [41.],
        [30.]], dtype=torch.float64)
tensor([[51.4234],
        [69.2569],
        [58.6057],
        [70.9119],
        [50.0224],
        [67.7363],
        [40.9583],
        [40.9038],
        [36.0853],
        [42.9509],
        [64.5266],
        [41.1832],
        [46.4910],
        [50.3956],
        [42.7205],
        [31.5594]])
tensor([[48.],
        [30.],
        [43.],
        [58.],
        [70.],
        [39.],
        [28.],
        [49.],
        [34.],
        [29.],
        [34.],
        [48.],
        [29.],
        [32.],
        [57.],
        [30.]], dtype=torch.float64)
tensor([[58.1993],
        [32.7801],
        [41.6715],
        [59.3292],
        [67.7300],
        [48.4527],
        [25.9687],
        [50.3198],
        [34.0857],
